## Kmeans 모델
- Compare  scikit learn vs onnx runtime 

In [4]:
# Iris Data Load 
# Train a model.

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn import cluster

import warnings
warnings.filterwarnings('ignore')
import time

# make_blobs data
X, y = make_blobs(n_samples = 300, n_features=2, centers=3, random_state=1)
print(X.shape, y.shape)

# KNN Model 생성 및 학습
kmeans = cluster.KMeans(n_clusters=3)
kmeans.fit(X)
kmeans

(300, 2) (300,)


KMeans(n_clusters=3)

In [6]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

import onnxruntime as rt
import numpy as np

# onnx 모델 변환
initial_type = [('float_input', FloatTensorType([None, 2]))]
onx = convert_sklearn(kmeans, initial_types=initial_type)

# onnx 모델 지정
with open("kmeans.onnx", "wb") as f:
    f.write(onx.SerializeToString())

## Inference time 비교

In [25]:
# scikit learn score

start = time.time()

# 1000번 predict
for i in range(1000):
    pred_skl = kmeans.predict(X)
end = time.time()

print('skl 걸린시간:', np.double(end-start))

# Compute the prediction with ONNX Runtime
sess = rt.InferenceSession('kmeans.onnx')
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

start = time.time()

# 1000번 predict
for i in range(1000):
    pred_onx = sess.run([label_name], {input_name: X.astype(np.float32)})[0]
end = time.time()

print('onnxruntime 걸린시간:', np.double(end-start))


skl 걸린시간: 0.30219197273254395
onnxruntime 걸린시간: 0.0827786922454834


## 군집 결과 비교

In [14]:
# Predict에 대한 군집 결과 비교
print('cluster-skl\n', pred_skl) 
print('clsuter-onnx\n', pred_onx) 


cluster-skl
 [1 2 1 0 0 0 1 0 0 2 2 2 1 1 2 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 2 2 1 2 1 2
 0 2 0 0 2 0 0 1 1 0 2 1 1 2 1 1 1 0 1 0 2 0 1 1 2 2 0 0 2 2 0 2 1 0 0 0 1
 0 1 2 0 0 1 1 1 0 1 0 2 1 1 0 1 2 0 1 2 1 2 1 1 2 2 1 2 0 2 2 0 0 1 2 1 2
 1 2 0 1 0 2 1 1 0 1 0 2 1 2 0 0 1 2 1 2 1 0 2 2 1 2 1 1 1 2 0 0 2 1 2 0 1
 2 0 2 1 2 1 0 2 2 1 2 1 1 0 0 2 0 1 2 0 0 1 0 1 1 0 0 0 2 1 2 2 0 1 0 1 0
 0 0 2 1 2 1 0 0 0 0 1 0 1 0 0 2 2 1 2 2 0 2 1 2 1 0 2 2 0 2 0 2 1 0 0 1 2
 1 0 2 0 1 0 0 2 0 1 2 2 0 2 1 2 2 0 2 0 2 1 0 2 0 0 0 1 0 2 1 2 0 2 1 2 1
 0 1 2 0 1 2 0 0 1 1 2 2 0 2 2 0 2 2 2 0 2 0 2 1 2 2 1 2 1 2 2 1 1 0 1 1 0
 1 2 2 2]
clsuter-onnx
 [1 2 1 0 0 0 1 0 0 2 2 2 1 1 2 0 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 2 2 1 2 1 2
 0 2 0 0 2 0 0 1 1 0 2 1 1 2 1 1 1 0 1 0 2 0 1 1 2 2 0 0 2 2 0 2 1 0 0 0 1
 0 1 2 0 0 1 1 1 0 1 0 2 1 1 0 1 2 0 1 2 1 2 1 1 2 2 1 2 0 2 2 0 0 1 2 1 2
 1 2 0 1 0 2 1 1 0 1 0 2 1 2 0 0 1 2 1 2 1 0 2 2 1 2 1 1 1 2 0 0 2 1 2 0 1
 2 0 2 1 2 1 0 2 2 1 2 1 1 0 0 2 0 1 2 0 0 1 0 1 1 0 0 0 2 1 2 